In [1]:
from time import sleep
from datetime import datetime
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import pickle
import ast
import shap
import matplotlib.pyplot as plt
from joblib import dump, load
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from hypermapper import optimizer
%matplotlib inline

In [2]:
concated_df = pd.read_csv('../../concated.csv')

In [3]:
df_cleaned = concated_df.fillna(0)
df_train, df_test = train_test_split(df_cleaned, test_size=0.99, random_state=42) # sample the dataset due to extended training time


In [4]:
df_X_train = df_train.iloc[:, :135]
df_X_test = df_test.iloc[:, :135]

df_Y_train = df_train.iloc[:, [135]].copy()
df_Y_train['1'] = df_train['UserID']
df_Y_test = df_test.iloc[:, [135]].copy()
df_Y_test['1'] = df_test['UserID']

In [5]:
# selected_columns = [0, 3, 4, 45, 57, 70, 96, 122, 123, 132]
selected_columns = [0, 3, 4, 45, 57, 70, 96, 106, 107, 116, 122, 123, 128, 132]

print(len(selected_columns))
df_X_train = df_X_train.iloc[:, selected_columns]
df_X_test = df_X_test.iloc[:, selected_columns]

14


In [6]:
threat_model = RandomForestClassifier(random_state=42)
threat_model.fit(df_X_train, df_Y_train.iloc[:, 1].to_numpy())
source_model = RandomForestClassifier(random_state=42)
source_model.fit(df_X_train, df_Y_train.iloc[:, 0].to_numpy())
threat_pred_new = threat_model.predict(df_X_test)
source_pred_new = source_model.predict(df_X_test)
threat_score = accuracy_score(df_Y_test.iloc[:, 1].to_numpy(), threat_pred_new)  # Source IP re-id classifier (Threat)
source_score = accuracy_score(df_Y_test.iloc[:, 0].to_numpy(), source_pred_new) # Network Intrusion classifier
print(threat_score, source_score)

0.7033603115625298 0.8398536529823607


#### only use top 10 IP addresses to train
value_counts = df_Y_train.iloc[:, 1].value_counts()

top_10_values = value_counts.index[:10]

mask = df_Y_train.iloc[:, 1].isin(top_10_values)

top10_df_Y_train = df_Y_train[mask]
top10_df_X_train = df_X_train[mask]

In [7]:
from sklearn.metrics import accuracy_score
from joblib import dump, load
def get_accuracy_for_feat_subset(df_X_train, df_X_test, df_Y_train, df_Y_test, feat_idx, custom_text_ext):
    print(feat_idx)
    if not feat_idx:
        return 1, 0.00001
    new_train = df_X_train.to_numpy()[:, feat_idx]
    new_test = df_X_test.to_numpy()[:, feat_idx]
    threat_model = RandomForestClassifier(random_state=42)
    threat_model.fit(new_train, df_Y_train.iloc[:, 1].to_numpy())
    source_model = RandomForestClassifier(random_state=42)
    source_model.fit(new_train, df_Y_train.iloc[:, 0].to_numpy())
    threat_pred_new = threat_model.predict(new_test)
    source_pred_new = source_model.predict(new_test)
    threat_score = accuracy_score(df_Y_test.iloc[:, 1].to_numpy(), threat_pred_new)  # Source IP re-id classifier (Threat)
    source_score = accuracy_score(df_Y_test.iloc[:, 0].to_numpy(), source_pred_new) # Network Intrusion classifier
    return threat_score, source_score

In [ ]:
parameters_file = "hypermapper-param.json"
def bayesian_opt(X):
    feat_idx = []
    for i in range(1, 15):
        if X[f'f{i}'] == 'true':
            feat_idx.append(i-1)
    threat_score, source_score = get_accuracy_for_feat_subset(df_X_train, df_X_test, df_Y_train, df_Y_test, feat_idx, 'bayesian')
    print("Source accuracy and threat accuracy:", source_score, threat_score)
    return threat_score/source_score
value_at_optimum = optimizer.optimize(parameters_file, bayesian_opt)

Design of experiment phase, number of new doe samples = 16384 .......
[0, 1, 3, 4, 6, 11, 12]
Source accuracy and threat accuracy: 0.8124188155139922 0.6521864913662695
[]
Source accuracy and threat accuracy: 1e-05 1
[13]
Source accuracy and threat accuracy: 0.4270669947739847 0.26867853151061816
[12]
Source accuracy and threat accuracy: 0.426850890095667 0.2688423527990203
[12, 13]
Source accuracy and threat accuracy: 0.4274608629780154 0.26898874629078395
[11]
Source accuracy and threat accuracy: 0.43072915577429377 0.26238825877721933
[11, 13]
Source accuracy and threat accuracy: 0.4165429293105331 0.27330038317915545
[11, 12]
Source accuracy and threat accuracy: 0.4151823993196188 0.27308544035394694
[11, 12, 13]
Source accuracy and threat accuracy: 0.4186110278449716 0.27333059135999555
[10]
Source accuracy and threat accuracy: 0.4274411114751584 0.2616632624370566
[10, 13]
Source accuracy and threat accuracy: 0.4191396710096736 0.2732829553825169
[10, 12]
Source accuracy and thre